In [2]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd

In [107]:
# Connection DB
connectionDB = 'sqlite:///./datasets/publications.db'
engineDB = create_engine(connectionDB)
inspector_db = inspect(engineDB)

In [4]:
# Tables DB
inspector_db.get_table_names()

['authors',
 'discounts',
 'employee',
 'jobs',
 'pub_info',
 'publishers',
 'roysched',
 'sales',
 'stores',
 'titleauthor',
 'titles']

In [5]:
print([i['name'] for i in inspector_db.get_columns('titleauthor')])
print([i['name'] for i in inspector_db.get_columns('titles')])
print([i['name'] for i in inspector_db.get_columns('sales')])

['au_id', 'title_id', 'au_ord', 'royaltyper']
['title_id', 'title', 'type', 'pub_id', 'price', 'advance', 'royalty', 'ytd_sales', 'notes', 'pubdate']
['stor_id', 'ord_num', 'ord_date', 'qty', 'payterms', 'title_id']


## Challenge 1 - Most Profiting Authors

In [67]:
#* Title ID
#* Author ID
#* Advance of each title and author
#   * The formula is:
#      ```
#      advance = titles.advance * titleauthor.royaltyper / 100
#      ```
#* Royalty of each sale
#    * The formula is:
#        ```
#        sales_royalty = titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100
#        ```
#    * Note that `titles.royalty` and `titleauthor.royaltyper` are divided by 100 respectively because they are percentage numbers instead of floats.

query_step1 = '''
SELECT t.title_id "Title ID", ta.au_id "Author ID", 
    t.advance * ta.royaltyper / 100 "Advance", 
    t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100 "Royalty"
FROM titleauthor ta
    INNER JOIN titles t ON t.title_id = ta.title_id
    INNER JOIN sales s ON s.title_id = t.title_id
ORDER BY "Title ID", "Author ID"
'''

df_step1 = pd.read_sql_query(query_step1, engineDB)
df_step1

,Title ID,Author ID,Advance,Royalty
0,BU1032,213-46-8915,2000,3.998
1,BU1032,213-46-8915,2000,7.996
2,BU1032,409-56-7008,3000,5.997
3,BU1032,409-56-7008,3000,11.994
4,BU1111,267-41-2394,2000,11.950
5,BU1111,724-80-9391,3000,17.925
6,BU2075,213-46-8915,10125,25.116
7,BU7832,274-80-9391,5000,29.985
8,MC2222,712-45-1867,0,23.988
9,MC3021,722-51-5454,11250,8.073


In [68]:
query_step2 = '''
SELECT t.title_id "Title ID", ta.au_id "Author ID", 
    SUM(t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) "Royalty"
FROM titleauthor ta
    INNER JOIN titles t ON t.title_id = ta.title_id
    INNER JOIN sales s ON s.title_id = t.title_id
GROUP BY t.title_id, ta.au_id
ORDER BY "Title ID", "Author ID"
'''

df_step2 = pd.read_sql_query(query_step2, engineDB)
df_step2

,Title ID,Author ID,Royalty
0,BU1032,213-46-8915,11.994
1,BU1032,409-56-7008,17.991
2,BU1111,267-41-2394,11.950
3,BU1111,724-80-9391,17.925
4,BU2075,213-46-8915,25.116
5,BU7832,274-80-9391,29.985
6,MC2222,712-45-1867,23.988
7,MC3021,722-51-5454,21.528
8,MC3021,899-46-2035,7.176
9,PC1035,238-95-7766,110.160


In [69]:
query_step2 = '''
SELECT "Title ID", "Author ID", SUM("Royalty") "Royalty"
FROM
(
SELECT t.title_id "Title ID", ta.au_id "Author ID", 
    t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100 "Royalty"
FROM titleauthor ta
    INNER JOIN titles t ON t.title_id = ta.title_id
    INNER JOIN sales s ON s.title_id = t.title_id
)
GROUP BY "Title ID", "Author ID"
ORDER BY "Title ID", "Author ID"
'''

df_step2 = pd.read_sql_query(query_step2, engineDB)
df_step2

,Title ID,Author ID,Royalty
0,BU1032,213-46-8915,11.994
1,BU1032,409-56-7008,17.991
2,BU1111,267-41-2394,11.950
3,BU1111,724-80-9391,17.925
4,BU2075,213-46-8915,25.116
5,BU7832,274-80-9391,29.985
6,MC2222,712-45-1867,23.988
7,MC3021,722-51-5454,21.528
8,MC3021,899-46-2035,7.176
9,PC1035,238-95-7766,110.160


In [70]:
query_step2 = '''
WITH Title_Au_Roy AS
(
SELECT t.title_id "Title ID", ta.au_id "Author ID", 
    SUM(t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) "Royalty"
FROM titleauthor ta
    INNER JOIN titles t ON t.title_id = ta.title_id
    INNER JOIN sales s ON s.title_id = t.title_id
GROUP BY t.title_id, ta.au_id
)
SELECT "Title ID", "Author ID", "Royalty"
FROM Title_Au_Roy
ORDER BY "Title ID", "Author ID"
'''

df_step2 = pd.read_sql_query(query_step2, engineDB)
df_step2

,Title ID,Author ID,Royalty
0,BU1032,213-46-8915,11.994
1,BU1032,409-56-7008,17.991
2,BU1111,267-41-2394,11.950
3,BU1111,724-80-9391,17.925
4,BU2075,213-46-8915,25.116
5,BU7832,274-80-9391,29.985
6,MC2222,712-45-1867,23.988
7,MC3021,722-51-5454,21.528
8,MC3021,899-46-2035,7.176
9,PC1035,238-95-7766,110.160


In [85]:
query_step3 = '''
WITH Title_Au_Roy AS
(
SELECT ta.au_id "Author ID", 
    SUM(t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) "Royalty"
FROM titleauthor ta
    INNER JOIN titles t ON t.title_id = ta.title_id
    INNER JOIN sales s ON s.title_id = t.title_id
GROUP BY ta.au_id
),
Title_Au_Adv AS
(
SELECT ta.au_id "Author ID", 
    SUM(t.advance * ta.royaltyper / 100) "Advance"
FROM titleauthor ta
    INNER JOIN titles t ON t.title_id = ta.title_id
GROUP BY ta.au_id
)
SELECT tar."Author ID", "Royalty" + "Advance" "Profits"
FROM Title_Au_Roy tar
    INNER JOIN Title_Au_Adv taa ON tar."Author ID" = taa."Author ID" 
ORDER BY tar."Author ID"
'''

df_step3= pd.read_sql_query(query_step3, engineDB)
df_step3

,Author ID,Profits
0,172-32-1176,2029.985
1,213-46-8915,12162.110
2,238-95-7766,7110.160
3,267-41-2394,4420.944
4,274-80-9391,5029.985
5,409-56-7008,3017.991
6,427-17-2319,4050.000
7,472-27-2349,2408.994
8,486-29-1786,4019.975
9,648-92-1872,4033.460


## Challenge 2 - Alternative Solution

In [87]:
query_challenge2 = '''
CREATE TEMPORARY TABLE profitsauthor AS
WITH Title_Au_Roy AS
(
SELECT ta.au_id "Author ID", 
    SUM(t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) "Royalty"
FROM titleauthor ta
    INNER JOIN titles t ON t.title_id = ta.title_id
    INNER JOIN sales s ON s.title_id = t.title_id
GROUP BY ta.au_id
),
Title_Au_Adv AS
(
SELECT ta.au_id "Author ID", 
    SUM(t.advance * ta.royaltyper / 100) "Advance"
FROM titleauthor ta
    INNER JOIN titles t ON t.title_id = ta.title_id
GROUP BY ta.au_id
)
SELECT tar."Author ID", "Royalty" + "Advance" "Profits"
FROM Title_Au_Roy tar
    INNER JOIN Title_Au_Adv taa ON tar."Author ID" = taa."Author ID" 
'''


## Challenge 3

In [105]:
# * `au_id` - Author ID
# * `profits` - The profits of the author aggregating the advances and royalties
query_challenge3 = '''
WITH Title_Au_Roy AS
(
SELECT ta.au_id "Author ID", 
    SUM(t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) "Royalty"
FROM titleauthor ta
    INNER JOIN titles t ON t.title_id = ta.title_id
    INNER JOIN sales s ON s.title_id = t.title_id
GROUP BY ta.au_id
),
Title_Au_Adv AS
(
SELECT ta.au_id "Author ID", 
    SUM(t.advance * ta.royaltyper / 100) "Advance"
FROM titleauthor ta
    INNER JOIN titles t ON t.title_id = ta.title_id
GROUP BY ta.au_id
)
SELECT tar."Author ID" au_id, "Royalty" + "Advance" profits
FROM Title_Au_Roy tar
    INNER JOIN Title_Au_Adv taa ON tar."Author ID" = taa."Author ID" 
ORDER BY Profits DESC
LIMIT 3
'''

df_challenge3 = pd.read_sql_query(query_challenge3, engineDB)
df_challenge3

,au_id,profits
0,213-46-8915,12162.110
1,722-51-5454,11271.528
2,998-72-3567,7224.956


In [104]:
# * `au_id` - Author ID
# * `profits` - The profits of the author aggregating the advances and royalties

query_challenge3 = '''
WITH Title_Au_Roy AS
(
SELECT ta.au_id "Author ID", 
    SUM(t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) "Royalty"
FROM titleauthor ta
    INNER JOIN titles t ON t.title_id = ta.title_id
    INNER JOIN sales s ON s.title_id = t.title_id
GROUP BY ta.au_id
),
Title_Au_Adv AS
(
SELECT ta.au_id "Author ID", 
    SUM(t.advance * ta.royaltyper / 100) "Advance"
FROM titleauthor ta
    INNER JOIN titles t ON t.title_id = ta.title_id
GROUP BY ta.au_id
),
au_profits AS
(
SELECT tar."Author ID", "Royalty" + "Advance" "Profits"
FROM Title_Au_Roy tar
    INNER JOIN Title_Au_Adv taa ON tar."Author ID" = taa."Author ID" 
ORDER BY Profits DESC
),
order_au_profits AS
(
SELECT "Author ID", "Profits",
    ROW_NUMBER() OVER() au_num
FROM au_profits
)
SELECT "Author ID" au_id, "Profits" profits FROM order_au_profits
WHERE order_au_profits.au_num <= 3
'''

df_challenge3 = pd.read_sql_query(query_challenge3, engineDB)
df_challenge3

,au_id,profits
0,213-46-8915,12162.110
1,722-51-5454,11271.528
2,998-72-3567,7224.956


In [109]:
query_view = '''
SELECT * FROM most_profiting_authors
'''

df_view = pd.read_sql_query(query_view, engineDB)
df_view

,au_id,profits
0,213-46-8915,12162.110
1,722-51-5454,11271.528
2,998-72-3567,7224.956
